In [1]:
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
import mxnet as mx

Using TensorFlow backend.
/home/ayush/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:12: DeprecationWarning: Importing from numpy.testing.decorators is deprecated, import from numpy.testing instead.
  from numpy.testing.decorators import setastest


In [2]:
CLEANING_PATTERN = re.compile("[\s\n\r\t.,:;\-_\'\"?!#&()\/%\[\]\{\}\<\>\\$@\!\*\+\=]")
VOCAB_SIZE = 5800
MAX_TIME = 15
LSTM_HIDDEN = 256

<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<input>:1: DeprecationWarning: invalid escape sequence \s
<ipython-input-2-e57eba91494f>:1: DeprecationWarning: invalid escape sequence \s
  CLEANING_PATTERN = re.compile("[\s\n\r\t.,:;\-_\'\"?!#&()\/%\[\]\{\}\<\>\\$@\!\*\+\=]")


In [3]:
x_train = []
x_test = []
with open('train.txt') as f:
    for line in f:
        x_train.append(line)

with open('test.txt') as f:
    for line in f:
        x_test.append(line)

In [4]:
tokenizer = Tokenizer(VOCAB_SIZE)

In [5]:
tokenizer.fit_on_texts(x_train)

In [6]:
train_x = tokenizer.texts_to_sequences(x_train)
test_x = tokenizer.texts_to_sequences(x_test)

In [7]:
sent_lenghts_train = [len(sent) for sent in train_x]
sent_lenghts_test = [len(sent) for sent in test_x]

In [8]:
train_x = pad_sequences(train_x,maxlen=MAX_TIME,padding='post')
test_x = pad_sequences(test_x,maxlen=MAX_TIME,padding='post')

In [9]:
wordIndex = tokenizer.word_index
wI = {}
for k,v in wordIndex.items():
    if v < VOCAB_SIZE:
        wI[k] = v
wordIndex = wI

In [50]:
train_y = np.zeros((len(train_x),MAX_TIME))
test_y = np.zeros((len(test_x),MAX_TIME))
train_y[:,:-1] = train_x[:,1:]
test_y[:,:-1] = test_x[:,1:]

In [79]:
vocab_size = VOCAB_SIZE
embedding_dim = 300
input_data = mx.sym.Variable('data')
label = mx.sym.Variable('softmax_label')
input_embed = mx.sym.Embedding(data=input_data,input_dim=vocab_size,output_dim=embedding_dim,\
                                   name='embed_weights')
lstm_cell = mx.rnn.LSTMCell(num_hidden=LSTM_HIDDEN)
begin_state = lstm_cell.begin_state()
output, states = lstm_cell.unroll(MAX_TIME,input_embed, begin_state,merge_outputs=True)
pred = mx.sym.Reshape(output, shape=(-1, LSTM_HIDDEN))
pred = mx.sym.FullyConnected(data=pred, num_hidden=vocab_size, name='pred')
pred = mx.sym.Reshape(pred, shape=(-1, vocab_size))

label = mx.sym.Reshape(label, shape=(-1,))
logits = mx.sym.log_softmax(pred, axis=-1)
loss = -mx.sym.pick(logits, label, axis=-1, keepdims=True)
loss = mx.sym.mean(loss, axis=0, exclude=True)
loss = mx.sym.make_loss(loss, name='nll')

In [80]:
batch_size=10
arg_params = {}
model = mx.module.Module(loss)

In [82]:
model.bind(data_shapes=[('data',(batch_size,MAX_TIME))], label_shapes=[('softmax_label', (batch_size,MAX_TIME))])
model.init_params(initializer=mx.initializer.Uniform(0.1))
model.init_optimizer('rmsprop')
#optimizer='rmsprop',num_epoch=10,learning_rate=0.005,\
#                                 numpy_batch_size=batch_size,initializer=mx.initializer.Uniform(0.1),\
#                                 arg_params=arg_params
bt  =mx.io.DataBatch(data=[tx[0:10]], label=[ty[0:10]])
model.forward(bt)
model.backward()
model.update()
outputs = model.get_outputs(merge_multi_context=False)[-1]
print(mx.nd.sum(outputs[0]).asscalar())

/home/ayush/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  


In [72]:
tx, ty = mx.nd.array(train_x), mx.nd.array(train_y)

In [51]:
train_y.shape

(3610, 15)

In [49]:
len(train_x)

3610

1242.103
